# 07 - script22_output_comparison

This notebook __compares the pobtot__ (or any other given pop. col) __for every city__ in metro_gdf_2020 by using:

* f'censo_inegi_{year[2:]}_ageb'
* f'pobvoronoi_inegi_{year[2:]}_mzaageb_mza' #From Script 22
* f'pobvoronoi_inegi_{year[2:]}_mzaageb_node' #From Script 22
* f'pobvoronoi_inegi_{year[2:]}_mzaageb_hex' #From Script 22

__RESULTS:__
* 2010 comparison is a __success__ (Biggest diff is Laguna, -3.39 people)
* 2020 comparison is a __success__ (Diffs can be attributed to blocks 800, confirmed using Aguascalientes)

## Import libraries

In [1]:
main_folder_path = '../../../'

In [2]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

from scipy.spatial import Voronoi, voronoi_plot_2d
import shapely

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(main_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Notebook config

In [3]:
# --------------- Database
# Cities
metro_schema = 'metropolis'
metro_table = 'metro_gdf_2020'
# Population data
pop_schema = 'sociodemografico'

## Load data

### Load all cities

In [4]:
metro_query = f"SELECT * FROM {metro_schema}.{metro_table}"
metro_gdf = aup.gdf_from_query(metro_query, geometry_col='geometry')

# Show
print(metro_gdf.shape)
metro_gdf.head(2)

(367, 6)


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry,city
0,01001,01,001,Aguascalientes,"POLYGON ((-102.10641 22.06035, -102.10368 22.0...",Aguascalientes
1,01005,01,005,Jesús María,"POLYGON ((-102.33568 22.05067, -102.33348 22.0...",Aguascalientes


## __2010 comparison__ [Success]

In [15]:
# Set comparison year (And tables)
year = 2010
# Population data - AGEBs
agebs_table = f'censo_inegi_{year[2:]}_ageb'
# Population data - Script 22 blocks
blocks_save_table = f'pobvoronoi_inegi_{year[2:]}_mzaageb_mza'
# Population data - Script 22 nodes
nodes_save_table = f'pobvoronoi_inegi_{year[2:]}_mzaageb_node'
# Population data - Script 22 hexs
hexs_save_table = f'pobvoronoi_inegi_{year[2:]}_mzaageb_hex'

# Prepare for comparison
comparison_df = pd.DataFrame()
i = 0
compare_col = 'pobtot'

# Iterate over each city loading different pop data and adding totals to comparison
city_lst = list(metro_gdf.city.unique()) #['Aguascalientes'] 
for city in city_lst:
    
     # --------------- SELECT CITY
    city_gdf = metro_gdf.loc[metro_gdf.city==city].copy()
    city_gdf = city_gdf.set_crs("EPSG:4326")

    # --------------- LOAD DATA

    # Load city's AGEBs
    print(f"{city} - Loading AGEBs.")
    # Create a tupple from a list with all unique cvegeo_mun ('CVE_ENT'+'CVE_MUN') of current city
    city_gdf['cvegeo_mun'] = city_gdf['CVE_ENT']+city_gdf['CVE_MUN']
    cvegeo_mun_lst = list(city_gdf.cvegeo_mun.unique())
    # To avoid error that happens when there's only one MUN in State: 
    # e.g.: <<< SELECT * FROM sociodemografico.censo_inegi_{year[2:]}_mza WHERE ("entidad" = '02') AND "mun" IN ('001',) >>>
    # Duplicate mun inside tupple if there's only one MUN.
    if len(cvegeo_mun_lst) >= 2:
        cvegeo_mun_tpl = str(tuple(cvegeo_mun_lst))
    else:
        cvegeo_mun_lst.append(cvegeo_mun_lst[0])
        cvegeo_mun_tpl = str(tuple(cvegeo_mun_lst))
    ageb_query = f"SELECT * FROM {pop_schema}.{agebs_table} WHERE \"cvegeo_mun\" IN {cvegeo_mun_tpl}"
    pop_ageb_gdf = aup.gdf_from_query(ageb_query, geometry_col='geometry')
    pop_ageb_gdf = pop_ageb_gdf.set_crs("EPSG:4326")

    # Load Script 22's blocks
    print(f"{city} - Loading Script 22's blocks.")
    blocks_query = f"SELECT * FROM {pop_schema}.{blocks_save_table} WHERE \"city\" LIKE \'{city}\'"
    s22_blocks_gdf = aup.gdf_from_query(blocks_query, geometry_col='geometry')

    # Load Script 22's nodes
    print(f"{city} - Loading Script 22's nodes.")
    nodes_query = f"SELECT * FROM {pop_schema}.{nodes_save_table} WHERE \"city\" LIKE \'{city}\'"
    s22_nodes_gdf = aup.gdf_from_query(nodes_query, geometry_col='geometry')

    # Load Script 22's hexs
    print(f"{city} - Loading Script 22's hexs.")
    hexs_query = f"SELECT * FROM {pop_schema}.{hexs_save_table} WHERE \"city\" LIKE \'{city}\'"
    s22_hexs_gdf = aup.gdf_from_query(hexs_query, geometry_col='geometry')
    s22_hexsres8_gdf = s22_hexs_gdf.loc[s22_hexs_gdf.res==8].copy()
    s22_hexsres9_gdf = s22_hexs_gdf.loc[s22_hexs_gdf.res==9].copy()
    s22_hexsres10_gdf = s22_hexs_gdf.loc[s22_hexs_gdf.res==10].copy()
    del s22_hexs_gdf

    # --------------- REGISTER DATA
    print(f"{city} - Registering Script 22's pop data.")
    comparison_df.loc[i,'city'] = city
    
    agebs_value = pop_ageb_gdf[compare_col].sum()
    comparison_df.loc[i,'agebs'] = agebs_value
    
    blocks_value = pop_ageb_gdf[compare_col].sum()
    comparison_df.loc[i,'blocks'] = blocks_value
    
    nodes_value = pop_ageb_gdf[compare_col].sum()
    comparison_df.loc[i,'nodes'] = nodes_value
    
    hexsres8_value = s22_hexsres8_gdf[compare_col].sum()
    comparison_df.loc[i,'hexsres8'] = hexsres8_value
    
    hexsres9_value = s22_hexsres9_gdf[compare_col].sum()
    comparison_df.loc[i,'hexsres9'] = hexsres9_value
    
    hexsres10_value = s22_hexsres10_gdf[compare_col].sum()
    comparison_df.loc[i,'hexsres10'] = hexsres10_value
    
    i+=1

Aguascalientes - Loading AGEBs.
Aguascalientes - Loading Script 22's blocks.
Aguascalientes - Loading Script 22's nodes.
Aguascalientes - Loading Script 22's hexs.
Aguascalientes - Registering Script 22's pop data.
Ensenada - Loading AGEBs.
Ensenada - Loading Script 22's blocks.
Ensenada - Loading Script 22's nodes.
Ensenada - Loading Script 22's hexs.
Ensenada - Registering Script 22's pop data.
Mexicali - Loading AGEBs.
Mexicali - Loading Script 22's blocks.
Mexicali - Loading Script 22's nodes.
Mexicali - Loading Script 22's hexs.
Mexicali - Registering Script 22's pop data.
Tijuana - Loading AGEBs.
Tijuana - Loading Script 22's blocks.
Tijuana - Loading Script 22's nodes.
Tijuana - Loading Script 22's hexs.
Tijuana - Registering Script 22's pop data.
La Paz - Loading AGEBs.
La Paz - Loading Script 22's blocks.
La Paz - Loading Script 22's nodes.
La Paz - Loading Script 22's hexs.
La Paz - Registering Script 22's pop data.
Los Cabos - Loading AGEBs.
Los Cabos - Loading Script 22's b

In [20]:
comparison_df['agebs-blocks'] = round(comparison_df['agebs']-comparison_df['blocks'],2)
comparison_df['agebs-nodes'] = round(comparison_df['agebs']-comparison_df['nodes'],2)
comparison_df['agebs-hexsres8'] = round(comparison_df['agebs']-comparison_df['hexsres8'],2)
comparison_df['agebs-hexsres9'] = round(comparison_df['agebs']-comparison_df['hexsres9'],2)
comparison_df['agebs-hexsres10'] = round(comparison_df['agebs']-comparison_df['hexsres10'],2)

# Show first 35
comparison_df.head(35)

,city,agebs,blocks,nodes,hexsres8,hexsres9,hexsres10,agebs-blocks,agebs-nodes,agebs-hexsres8,agebs-hexsres9,agebs-hexsres10
0,Aguascalientes,828378.0,828378.0,828378.0,8.283780e+05,8.283780e+05,8.283780e+05,0.0,0.0,0.04,0.04,0.04
1,Ensenada,398122.0,398122.0,398122.0,3.981220e+05,3.981220e+05,3.981220e+05,0.0,0.0,-0.03,-0.03,-0.03
2,Mexicali,838116.0,838116.0,838116.0,8.381158e+05,8.381158e+05,8.381158e+05,0.0,0.0,0.24,0.24,0.24
3,Tijuana,1595559.0,1595559.0,1595559.0,1.595559e+06,1.595559e+06,1.595559e+06,0.0,0.0,-0.00,-0.01,-0.01
4,La Paz,225022.0,225022.0,225022.0,2.250220e+05,2.250220e+05,2.250220e+05,0.0,0.0,-0.02,-0.02,-0.02
5,Los Cabos,215545.0,215545.0,215545.0,2.155450e+05,2.155450e+05,2.155450e+05,0.0,0.0,0.02,0.02,0.02
6,Campeche,233864.0,233864.0,233864.0,2.338640e+05,2.338640e+05,2.338640e+05,0.0,0.0,-0.03,-0.03,-0.03
7,Laguna,1066672.0,1066672.0,1066672.0,1.066675e+06,1.066675e+06,1.066675e+06,0.0,0.0,-3.39,-3.38,-3.38
8,Monclova,330608.0,330608.0,330608.0,3.306080e+05,3.306080e+05,3.306080e+05,0.0,0.0,-0.01,-0.01,-0.01
9,Piedras Negras,177291.0,177291.0,177291.0,1.772910e+05,1.772910e+05,1.772910e+05,0.0,0.0,0.05,0.05,0.05


In [21]:
# Show last 36
comparison_df.tail(36)

,city,agebs,blocks,nodes,hexsres8,hexsres9,hexsres10,agebs-blocks,agebs-nodes,agebs-hexsres8,agebs-hexsres9,agebs-hexsres10
35,Cuautla,249448.0,249448.0,249448.0,2.494480e+05,2.494480e+05,2.494480e+05,0.0,0.0,0.00,0.00,0.00
36,Cuernavaca,884145.0,884145.0,884145.0,8.841450e+05,8.841450e+05,8.841450e+05,0.0,0.0,-0.00,-0.00,-0.00
37,Tepic,383232.0,383232.0,383232.0,3.832320e+05,3.832320e+05,3.832320e+05,0.0,0.0,-0.00,-0.00,-0.00
38,Monterrey,4131710.0,4131710.0,4131710.0,4.131710e+06,4.131710e+06,4.131710e+06,0.0,0.0,-0.03,-0.04,-0.04
39,Oaxaca,586495.0,586495.0,586495.0,5.864950e+05,5.864950e+05,5.864950e+05,0.0,0.0,0.00,0.01,0.00
40,Puebla,2290801.0,2290801.0,2290801.0,2.290801e+06,2.290801e+06,2.290801e+06,0.0,0.0,-0.02,-0.03,-0.03
41,San Martin,232728.0,232728.0,232728.0,2.327280e+05,2.327280e+05,2.327280e+05,0.0,0.0,-0.01,-0.01,-0.01
42,Tehuacan,277745.0,277745.0,277745.0,2.777450e+05,2.777450e+05,2.777450e+05,0.0,0.0,-0.01,-0.01,-0.01
43,Queretaro,920993.0,920993.0,920993.0,9.209930e+05,9.209930e+05,9.209930e+05,0.0,0.0,0.02,0.01,0.01
44,Cancun,674206.0,674206.0,674206.0,6.742060e+05,6.742060e+05,6.742060e+05,0.0,0.0,0.03,0.03,0.02


## __2020 comparison__ [Success]

In [5]:
# Set comparison year (And tables)
year = '2020'
# Population data - AGEBs
agebs_table = f'censo_inegi_{year[2:]}_ageb'
# Population data - Script 22 blocks
blocks_save_table = f'pobvoronoi_inegi_{year[2:]}_mzaageb_mza'
# Population data - Script 22 nodes
nodes_save_table = f'pobvoronoi_inegi_{year[2:]}_mzaageb_node'
# Population data - Script 22 hexs
hexs_save_table = f'pobvoronoi_inegi_{year[2:]}_mzaageb_hex'

# Prepare for comparison
comparison_df = pd.DataFrame()
i = 0
compare_col = 'pobtot'

# Iterate over each city loading different pop data and adding totals to comparison
city_lst = list(metro_gdf.city.unique()) #['Aguascalientes'] 
for city in city_lst:
    
     # --------------- SELECT CITY
    city_gdf = metro_gdf.loc[metro_gdf.city==city].copy()
    city_gdf = city_gdf.set_crs("EPSG:4326")

    # --------------- LOAD DATA

    # Load city's AGEBs
    print(f"{city} - Loading AGEBs.")
    # Create a tupple from a list with all unique cvegeo_mun ('CVE_ENT'+'CVE_MUN') of current city
    city_gdf['cvegeo_mun'] = city_gdf['CVE_ENT']+city_gdf['CVE_MUN']
    cvegeo_mun_lst = list(city_gdf.cvegeo_mun.unique())
    # To avoid error that happens when there's only one MUN in State: 
    # e.g.: <<< SELECT * FROM sociodemografico.censo_inegi_{year[2:]}_mza WHERE ("entidad" = '02') AND "mun" IN ('001',) >>>
    # Duplicate mun inside tupple if there's only one MUN.
    if len(cvegeo_mun_lst) >= 2:
        cvegeo_mun_tpl = str(tuple(cvegeo_mun_lst))
    else:
        cvegeo_mun_lst.append(cvegeo_mun_lst[0])
        cvegeo_mun_tpl = str(tuple(cvegeo_mun_lst))
    ageb_query = f"SELECT * FROM {pop_schema}.{agebs_table} WHERE \"cvegeo_mun\" IN {cvegeo_mun_tpl}"
    pop_ageb_gdf = aup.gdf_from_query(ageb_query, geometry_col='geometry')
    pop_ageb_gdf = pop_ageb_gdf.set_crs("EPSG:4326")

    # Load Script 22's blocks
    print(f"{city} - Loading Script 22's blocks.")
    blocks_query = f"SELECT * FROM {pop_schema}.{blocks_save_table} WHERE \"city\" LIKE \'{city}\'"
    s22_blocks_gdf = aup.gdf_from_query(blocks_query, geometry_col='geometry')

    # Load Script 22's nodes
    print(f"{city} - Loading Script 22's nodes.")
    nodes_query = f"SELECT * FROM {pop_schema}.{nodes_save_table} WHERE \"city\" LIKE \'{city}\'"
    s22_nodes_gdf = aup.gdf_from_query(nodes_query, geometry_col='geometry')

    # Load Script 22's hexs
    print(f"{city} - Loading Script 22's hexs.")
    hexs_query = f"SELECT * FROM {pop_schema}.{hexs_save_table} WHERE \"city\" LIKE \'{city}\'"
    s22_hexs_gdf = aup.gdf_from_query(hexs_query, geometry_col='geometry')
    s22_hexsres8_gdf = s22_hexs_gdf.loc[s22_hexs_gdf.res==8].copy()
    s22_hexsres9_gdf = s22_hexs_gdf.loc[s22_hexs_gdf.res==9].copy()
    s22_hexsres10_gdf = s22_hexs_gdf.loc[s22_hexs_gdf.res==10].copy()
    del s22_hexs_gdf

    # --------------- REGISTER DATA
    print(f"{city} - Registering Script 22's pop data.")
    comparison_df.loc[i,'city'] = city
    
    agebs_value = pop_ageb_gdf[compare_col].sum()
    comparison_df.loc[i,'agebs'] = agebs_value
    
    blocks_value = s22_blocks_gdf[compare_col].sum()
    comparison_df.loc[i,'blocks'] = blocks_value
    
    nodes_value = s22_nodes_gdf[compare_col].sum()
    comparison_df.loc[i,'nodes'] = nodes_value
    
    hexsres8_value = s22_hexsres8_gdf[compare_col].sum()
    comparison_df.loc[i,'hexsres8'] = hexsres8_value
    
    hexsres9_value = s22_hexsres9_gdf[compare_col].sum()
    comparison_df.loc[i,'hexsres9'] = hexsres9_value
    
    hexsres10_value = s22_hexsres10_gdf[compare_col].sum()
    comparison_df.loc[i,'hexsres10'] = hexsres10_value
    
    i+=1

Aguascalientes - Loading AGEBs.
Aguascalientes - Loading Script 22's blocks.
Aguascalientes - Loading Script 22's nodes.
Aguascalientes - Loading Script 22's hexs.
Aguascalientes - Registering Script 22's pop data.
Ensenada - Loading AGEBs.
Ensenada - Loading Script 22's blocks.
Ensenada - Loading Script 22's nodes.
Ensenada - Loading Script 22's hexs.
Ensenada - Registering Script 22's pop data.
Mexicali - Loading AGEBs.
Mexicali - Loading Script 22's blocks.
Mexicali - Loading Script 22's nodes.
Mexicali - Loading Script 22's hexs.
Mexicali - Registering Script 22's pop data.
Tijuana - Loading AGEBs.
Tijuana - Loading Script 22's blocks.
Tijuana - Loading Script 22's nodes.
Tijuana - Loading Script 22's hexs.
Tijuana - Registering Script 22's pop data.
La Paz - Loading AGEBs.
La Paz - Loading Script 22's blocks.
La Paz - Loading Script 22's nodes.
La Paz - Loading Script 22's hexs.
La Paz - Registering Script 22's pop data.
Los Cabos - Loading AGEBs.
Los Cabos - Loading Script 22's b

In [6]:
comparison_df['agebs-blocks'] = round(comparison_df['agebs']-comparison_df['blocks'],2)
comparison_df['agebs-nodes'] = round(comparison_df['agebs']-comparison_df['nodes'],2)
comparison_df['agebs-hexsres8'] = round(comparison_df['agebs']-comparison_df['hexsres8'],2)
comparison_df['agebs-hexsres9'] = round(comparison_df['agebs']-comparison_df['hexsres9'],2)
comparison_df['agebs-hexsres10'] = round(comparison_df['agebs']-comparison_df['hexsres10'],2)

# Show first 35
comparison_df.head(35)

,city,agebs,blocks,nodes,hexsres8,hexsres9,hexsres10,agebs-blocks,agebs-nodes,agebs-hexsres8,agebs-hexsres9,agebs-hexsres10
0,Aguascalientes,1042295.0,1043905.0,1.043905e+06,1.043905e+06,1.043905e+06,1.043905e+06,-1610.0,-1609.98,-1609.97,-1609.97,-1609.98
1,Ensenada,397099.0,396709.0,3.967090e+05,3.967090e+05,3.967090e+05,3.967090e+05,390.0,389.98,389.98,389.98,389.98
2,Mexicali,950970.0,959257.0,9.592567e+05,9.592567e+05,9.592567e+05,9.592567e+05,-8287.0,-8286.73,-8286.73,-8286.73,-8286.73
3,Tijuana,2000259.0,2002469.0,2.002469e+06,2.002469e+06,2.002469e+06,2.002469e+06,-2210.0,-2210.03,-2210.03,-2210.03,-2210.03
4,La Paz,270846.0,270681.0,2.706810e+05,2.706810e+05,2.706810e+05,2.706810e+05,165.0,165.00,165.00,165.00,165.00
5,Los Cabos,338979.0,338675.0,3.386750e+05,3.386750e+05,3.386750e+05,3.386750e+05,304.0,304.00,304.00,304.00,304.00
6,Campeche,267558.0,267387.0,2.673870e+05,2.673870e+05,2.673870e+05,2.673870e+05,171.0,171.02,171.02,171.02,171.02
7,Laguna,1218759.0,1218686.0,1.218686e+06,1.218686e+06,1.218686e+06,1.218686e+06,73.0,73.07,73.07,73.07,73.07
8,Monclova,364881.0,367434.0,3.674340e+05,3.674340e+05,3.674340e+05,3.674340e+05,-2553.0,-2553.01,-2553.01,-2553.01,-2553.01
9,Piedras Negras,206172.0,206163.0,2.061630e+05,2.061630e+05,2.061630e+05,2.061630e+05,9.0,9.00,9.00,9.00,9.00


In [7]:
# Show last 36
comparison_df.tail(36)

,city,agebs,blocks,nodes,hexsres8,hexsres9,hexsres10,agebs-blocks,agebs-nodes,agebs-hexsres8,agebs-hexsres9,agebs-hexsres10
35,Cuautla,269827.0,274340.0,2.743400e+05,2.743400e+05,2.743400e+05,2.743400e+05,-4513.0,-4513.00,-4513.00,-4513.00,-4513.00
36,Cuernavaca,983015.0,991976.0,9.919760e+05,9.919760e+05,9.919760e+05,9.919760e+05,-8961.0,-8961.03,-8961.03,-8961.03,-8961.03
37,Tepic,438004.0,443405.0,4.434050e+05,4.434050e+05,4.434050e+05,4.434050e+05,-5401.0,-5401.00,-5401.01,-5401.00,-5401.00
38,Monterrey,5245560.0,5245269.0,5.245269e+06,5.245269e+06,5.245269e+06,5.245269e+06,291.0,290.72,290.71,290.72,290.72
39,Oaxaca,660807.0,662287.0,6.622870e+05,6.622870e+05,6.622870e+05,6.622870e+05,-1480.0,-1480.02,-1480.01,-1480.02,-1480.02
40,Puebla,2654156.0,2647940.0,2.647940e+06,2.647940e+06,2.647940e+06,2.647940e+06,6216.0,6216.04,6216.05,6216.04,6216.04
41,San Martin,287426.0,285397.0,2.853970e+05,2.853970e+05,2.853970e+05,2.853970e+05,2029.0,2028.99,2028.99,2028.99,2028.99
42,Tehuacan,338779.0,337788.0,3.377880e+05,3.377880e+05,3.377880e+05,3.377880e+05,991.0,990.98,990.98,990.98,990.98
43,Queretaro,1381392.0,1379350.0,1.379351e+06,1.379351e+06,1.379351e+06,1.379351e+06,2042.0,2041.01,2041.01,2041.01,2041.01
44,Cancun,928428.0,928339.0,9.283391e+05,9.283391e+05,9.283391e+05,9.283391e+05,89.0,88.89,88.88,88.89,88.89


### Find 2020 problem [Aguascalientes]

In [8]:
# Iterate over each city loading different pop data and adding totals to comparison
city_lst = ['Aguascalientes'] 
for city in city_lst:
    
     # --------------- SELECT CITY
    city_gdf = metro_gdf.loc[metro_gdf.city==city].copy()
    city_gdf = city_gdf.set_crs("EPSG:4326")

    # --------------- LOAD DATA

    # Load city's AGEBs
    print(f"{city} - Loading AGEBs.")
    # Create a tupple from a list with all unique cvegeo_mun ('CVE_ENT'+'CVE_MUN') of current city
    city_gdf['cvegeo_mun'] = city_gdf['CVE_ENT']+city_gdf['CVE_MUN']
    cvegeo_mun_lst = list(city_gdf.cvegeo_mun.unique())
    # To avoid error that happens when there's only one MUN in State: 
    # e.g.: <<< SELECT * FROM sociodemografico.censo_inegi_{year[2:]}_mza WHERE ("entidad" = '02') AND "mun" IN ('001',) >>>
    # Duplicate mun inside tupple if there's only one MUN.
    if len(cvegeo_mun_lst) >= 2:
        cvegeo_mun_tpl = str(tuple(cvegeo_mun_lst))
    else:
        cvegeo_mun_lst.append(cvegeo_mun_lst[0])
        cvegeo_mun_tpl = str(tuple(cvegeo_mun_lst))
    ageb_query = f"SELECT * FROM {pop_schema}.{agebs_table} WHERE \"cvegeo_mun\" IN {cvegeo_mun_tpl}"
    pop_ageb_gdf = aup.gdf_from_query(ageb_query, geometry_col='geometry')
    pop_ageb_gdf = pop_ageb_gdf.set_crs("EPSG:4326")

    # Load Script 22's blocks
    print(f"{city} - Loading Script 22's blocks.")
    blocks_query = f"SELECT * FROM {pop_schema}.{blocks_save_table} WHERE \"city\" LIKE \'{city}\'"
    s22_blocks_gdf = aup.gdf_from_query(blocks_query, geometry_col='geometry')

    # Load Script 22's nodes
    print(f"{city} - Loading Script 22's nodes.")
    nodes_query = f"SELECT * FROM {pop_schema}.{nodes_save_table} WHERE \"city\" LIKE \'{city}\'"
    s22_nodes_gdf = aup.gdf_from_query(nodes_query, geometry_col='geometry')

    # Load Script 22's hexs
    print(f"{city} - Loading Script 22's hexs.")
    hexs_query = f"SELECT * FROM {pop_schema}.{hexs_save_table} WHERE \"city\" LIKE \'{city}\'"
    s22_hexs_gdf = aup.gdf_from_query(hexs_query, geometry_col='geometry')
    s22_hexsres8_gdf = s22_hexs_gdf.loc[s22_hexs_gdf.res==8].copy()
    s22_hexsres9_gdf = s22_hexs_gdf.loc[s22_hexs_gdf.res==9].copy()
    s22_hexsres10_gdf = s22_hexs_gdf.loc[s22_hexs_gdf.res==10].copy()
    del s22_hexs_gdf

Aguascalientes - Loading AGEBs.
Aguascalientes - Loading Script 22's blocks.
Aguascalientes - Loading Script 22's nodes.
Aguascalientes - Loading Script 22's hexs.


In [10]:
pop_ageb_gdf.head(2)

,nom_ent,nom_mun,nom_loc,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,p_3ymas,...,cve_ent,cve_mun,cve_loc,cve_ageb,cve_mza,cvegeo_mun,cvegeo_loc,cvegeo_ageb,cvegeo_mza,geometry
0,Aguascalientes,Aguascalientes,Total AGEB urbana,2237,1137.0,1100.0,216.0,103.0,113.0,2021.0,...,01,001,0001,0017,000,01001,010010001,0100100010017,0100100010017000,"MULTIPOLYGON (((-102.25467 21.94536, -102.2549..."
1,Aguascalientes,Aguascalientes,Total AGEB urbana,1411,712.0,699.0,45.0,24.0,21.0,1366.0,...,01,001,0001,006A,000,01001,010010001,010010001006A,010010001006A000,"MULTIPOLYGON (((-102.28214 21.94904, -102.2793..."


In [16]:
# Sum blocks'pop by ageb
grouped_blocks = s22_blocks_gdf.groupby('cvegeo_ageb').agg({'pobtot':np.sum})
grouped_blocks.reset_index(inplace=True)
grouped_blocks.rename(columns={'pobtot':'blockpop'},inplace=True)

# Merge with ageb data
merged = pd.merge(pop_ageb_gdf[['pobtot','cvegeo_ageb']],grouped_blocks[['blockpop','cvegeo_ageb']],on='cvegeo_ageb')

# Find population differences
merged['pop_diff'] = merged['pobtot']-merged['blockpop']
merged.loc[merged.pop_diff!=0]

,pobtot,cvegeo_ageb,blockpop,pop_diff
101,2001,0100100010888,1999,2
242,5275,0100100013721,5266,9
276,1237,0100100014113,1220,17
307,807,0100102374594,763,44
311,2351,0100102933929,2276,75
317,1501,0100103674607,1464,37
318,1115,0100103674611,1025,90
327,1998,010010237458A,1976,22
342,5218,0100500010497,4561,657
353,1638,0100500010656,1633,5


### NOTE: Analysis confirmed the three AGEBs with the greatest pop diff between pobtot (ageb data) and blockpop (script 22 data) comes from blocks 800.
* AGEB 100500010497 has a block 800 with 657 pobtot.
* AGEB 0101100110206 has a block 800 with 200 pobtot.
* AGEB 0100103674611 has a block 800 with 90 pobtot.